# 1.Mixtures of Gaussians

given a training set $\left\{x^{(1)},...,x^{(n)}\right\}$ with no labels. each point $x^{(i)}$ has a latent variable $z^{(i)}$, the data is specified by a joint probability $p(x^{(i)}, z^{(i)}) = p(x^{(i)}| z^{(i)})p(z^{(i)})$

mixtures of gaussians assumes $z^{(i)}\sim Multinomial(\phi)$ (where $\phi_{j} \ge 0, \sum_{j=1}^{k}\phi_{j}=1$, $\phi_{j}$ gives $p(z^{(i)}=j)$), and $x^{(i)}|z^{(i)}=j \sim \mathcal{N}(\mu_{j}, \Sigma_{j})$. we let $k$ denote the number of values $z^{(i)}$'s can take on.

thus our model posits that each $x^{(i)}$ was generated by randomly choosing $z^{(i)}$ from $\{1,...,k\}$, and $x^{(i)}$ was drawn from one of $k$ gaussians depending on $z^{(i)}$.

the parameters of our model are $\phi, \mu, \Sigma$, to estimate them, we can write down the likelihood of our data:

$$
\begin{equation}
\begin{split}
l(\phi, \mu, \Sigma) =& \sum_{i=1}^{n}log\,p(x^{(i)};\phi,\mu,\Sigma)\\
=& \sum_{i=1}^{n}log\sum_{z^{(i)}=1}^{k}p(x^{(i)}|z^{(i)}; \mu,\Sigma)p(z^{(i)};\phi)
\end{split}
\end{equation}
$$

it is not possible to find the maximum likelihood estimate of the parameters in closed form.

on the other hand, if we knew what $z^{(i)}$'s were, the maximum likelihood problem would have been easy.

specifically, we could then write down the likelihood as:

$$l(\phi, \mu, \Sigma) = \sum_{i=1}^{n}log\,p(x^{(i)}|z^{(i)};\mu,\Sigma) + log\,p(z^{(i)}; \phi)$$

maximizing this with respect to $\phi, \mu, \Sigma$ gives the parameters:

$$\phi_{j} = \frac{1}{n}\sum_{i=1}^{n}1\{z^{(i)} = j\}$$

$$\mu_{j} = \frac{\sum_{i=1}^{n}1\{z^{(i)} = j\}x^{(i)}}{\sum_{i=1}^{n}1\{z^{(i)} = j\}}$$

$$\Sigma_{j} = \frac{\sum_{i=1}^{n}1\{z^{(i)} = j\}(x^{(i)} - \mu_{j})(x^{(i)} - \mu_{j})^{T}}{\sum_{i=1}^{n}1\{z^{(i)} = j\}}$$

it is identical to the gaussian discriminant analysis(GDA).

however, in our density estimation problems, $z^{(i)}$'s are not known, what can we do?

the EM algorithm is an iterative algorithm that has two main steps:

**1.the E-step, it tries to "guess" the values of the $z^{(i)}$'s.**

**2.the M-step, it updates the parameters of our model based on our guesses.**

since in the M-step we are pretending that the guesses in the first part were correct, the maximization becomes easy, here's the algorithm:


(E-step) for each $i, j$ set

$$w_{j}^{(i)} := p(z^{i}=j|x^{(i)}; \phi,\mu,\Sigma)$$

(M-step) update the parameters:

$$\phi_{j} := \frac{1}{n}\sum_{i=1}^{n}w_{j}^{(i)}$$

$$\mu_{j} := \frac{\sum_{i=1}^{n}w_{j}^{(i)}x^{(i)}}{\sum_{i=1}^{n}w_{j}^{(i)}}$$

$$\Sigma_{j} := \frac{\sum_{i=1}^{n}w_{j}^{(i)}(x^{(i)} - \mu_{j})(x^{(i)} - \mu_{j})^{T}}{\sum_{i=1}^{n}w_{j}^{(i)}}$$

the EM-algorithm likes the K-means-algorithm.

except that instead of "hard" cluster assignments $c(i)$, we have the "soft" assignments $w_{j}^{(i)}$.

# 2.The EM algorithm

here comes the general view of EM.

like the assumption before, we have the training set $\left\{x^{(1)},...,x^{(n)}\right\}$, each point $x^{(i)}$ has a latent variable $z^{(i)}$.

denote $\theta$ as the distribution parameters($\phi,\mu,\Sigma$ in mixtures of gaussian), then density of $x$ can be obtained by:

$$p(x;\theta) = \sum_{z}p(x,z;\theta)$$

the log-likelihood of the data:

$$
\begin{equation}
\begin{split}
l(\theta) =& \sum_{i=1}^{n}log\,p(x^{(i)}; \theta) \\
=& \sum_{i=1}^{n}log\sum_{z^{(i)}}p(x^{(i)}, z^{(i)}; \theta)
\end{split}
\end{equation}
$$

instead of solving the maximum-likelihood directly(usually very hard), the EM algorithm repeatedly construct a lower-bound on $l$(E-step), and then optimize that lower-bound(M-step).

we first consider optimizing the likelihood $log\,p(x)$ for a single example $x$, optimize $log\,p(x;\theta)$:

$$log\,p(x;\theta) = log\sum_{z}p(x,z;\theta)$$

let $Q$ be a distribution over the possible values of $z$, then:

$$
\begin{equation}
\begin{split}
log\,p(x;\theta) =& log\sum_{z}p(x,z;\theta)\\
=& log\sum_{z}Q(z)\frac{p(x,z;\theta)}{Q(z)}\\
\ge& \sum_{z}Q(z)log\frac{p(x,z;\theta)}{Q(z)}
\end{split}
\end{equation}
$$

the last step uses Jensen's inequality, for $(log\,x)'' = -1/x^{2} < 0$ so that strictly concave.

for any distribution $Q$, the above formula gives a lower-bound on $log\,p(x;\theta)$.

to make the bound tight for a particular value of $\theta$, we want the Jensen's inequality to hold with equality. it is sufficient the expecation take on "constant"-valued random variables, i.e:

$$\frac{p(x,z;\theta)}{Q(z)} = c$$

this leads to:

$$
\begin{equation}
\begin{split}
Q(z) =& \frac{p(x,z;\theta)}{\sum_{z}p(x,z;\theta)}\\
=& \frac{p(x,z;\theta)}{p(x;\theta)}\\
=& p(z|x;\theta)
\end{split}
\end{equation}
$$

thus, we simply set $Q$ to be the posterior distribution of $z$'s given $x$ and $\theta$.

for convenience, we call the bound as **evidence lower bound(ELBO)**:

$$ELBO(x;Q,\theta) = \sum_{z}Q(z)log\frac{p(x,z;\theta)}{Q(z)}$$

the estimation now can re-write as:

$$\forall Q,\theta,x,\quad log\,p(x;\theta) \ge ELBO(x;Q,\theta)$$

Intuitively, the EM algorithm alternatively update $Q$ and $\theta$ by:

a.setting $Q(z) = p(z|x;\theta)$ so that $ELBO(x;Q,\theta) = log\,p(x;\theta)$

b.maximizing $ELBO(x;Q,\theta)$ with respect to $\theta$.

now consider multiple examples $\left\{x^{(1)},...,x^{(n)}\right\}$, note the optimal choice of $Q$ is $p(z|x;\theta)$, and it depends on the particular example $x$.

therefore we introduce $n$ distributions $Q_{1},...,Q_{n}$, one for each example $x^{(i)}$, for each example:

$$log\,p(x^{(i)};\theta) \ge ELBO(x^{(i)};Q_{i},\theta) = \sum_{z^{(i)}}Q_{i}(z^{(i)})log\frac{p(x^{(i)},z^{(i)};\theta)}{Q_{i}(z^{(i)})}$$

taking sum over all examples, we obtain a lower bound for the log-likelihood:

$$
\begin{equation}
\begin{split}
l(\theta) \ge& \sum_{i}ELBO(x^{(i)};Q_{i},\theta)\\
=& \sum_{i}\sum_{z^{(i)}}Q_{i}(z^{(i)})log\frac{p(x^{(i)},z^{(i)};\theta)}{Q_{i}(z^{(i)})}
\end{split}
\end{equation}
$$

the above inequality holds for **any** distributions $Q_{1},...,Q_{n}$. equality holds when:

$$Q_{i}(z^{(i)}) =  p(z^{(i)}|x^{(i)};\theta)$$

we now come to the definition of EM algorithm:

(E-step) for each $i$, set:

$$Q_{i}(z^{(i)}) :=  p(z^{(i)}|x^{(i)};\theta)$$

(M-step) for fixed $Q_{i}$'s, set:

$$
\begin{equation}
\begin{split}
\theta :=& \underset{\theta}{argmax}\sum_{i=1}^{n}ELBO(x^{(i)};Q_{i},\theta)\\
=& \underset{\theta}{argmax}\sum_{i=1}^{n}\sum_{z^{(i)}}Q_{i}(z^{(i)})log\frac{p(x^{(i)},z^{(i)};\theta)}{Q_{i}(z^{(i)})}
\end{split}
\end{equation}
$$

# 3.Convergence of EM algorithm

suppose $\theta^{(t)}$ and $\theta^{(t+1)}$ are the parameters from two successive iterations of EM, we will now prove that $l(\theta^{(t)}) \le l(\theta^{(t+1)})$.

$$
\begin{equation}
\begin{split}
l(\theta^{(t+1)}) \ge& \sum_{i=1}^{n}ELBO(x^{(i)};Q_{i}^{(t)};\theta^{(t+1)})\quad (\mbox{by Jensen's inequality})\\
\ge& \sum_{i=1}^{n}ELBO(x^{(i)};Q_{i}^{(t)};\theta^{(t)})\quad (\mbox{by M-step}) \\
=& l(\theta^{(t)})\quad (\mbox{by E-step})
\end{split}
\end{equation}
$$

if we define:

$$ELBO(Q, \theta) = \sum_{i=1}^{n}ELBO(x^{(i)}; Q_{i}, \theta)$$

then we know $l(\theta) \ge ELBO(Q, \theta)$.

the EM algorithm can be viewed an alternating maximization algorithm on $ELBO(Q, \theta)$:

the E-step maximizes it with respect to $Q$.

the M-steo maximizes it with respect to $\theta$.